In [1]:
# Load Libraries
import warnings
from datetime import datetime

import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, ELU, Embedding, BatchNormalization, Convolution1D, MaxPooling1D, concatenate
from keras.layers.core import Dense, Dropout, Lambda
from keras.models import Model
from keras.optimizers import Adam
from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
def conv_fully(max_len=73, emb_dim=32, max_vocab_len=40, W_reg=regularizers.l2(1e-4)):
    """CNN model with the Keras functional API"""

    # Input
    main_input = Input(shape=(max_len,), dtype='int32', name='main_input')

    # Embedding layer
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(main_input)
    emb = Dropout(0.2)(emb)

    def sum_1d(X):
        return K.sum(X, axis=1)

    def get_conv_layer(emb, kernel_size=5, filters=256):
        # Conv layer
        conv = Convolution1D(kernel_size=kernel_size, filters=filters, border_mode='same')(emb)
        conv = ELU()(conv)
        conv = MaxPooling1D()(conv)
        conv = Lambda(sum_1d, output_shape=(filters,))(conv)
        conv = Dropout(0.5)(conv)

        return conv

    # Multiple Conv Layers
    # 커널 사이즈를 다르게 한 conv
    conv1 = get_conv_layer(emb, kernel_size=2, filters=256)
    conv2 = get_conv_layer(emb, kernel_size=3, filters=256)
    conv3 = get_conv_layer(emb, kernel_size=4, filters=256)
    conv4 = get_conv_layer(emb, kernel_size=5, filters=256)

    # Fully Connected Layers
    # 위 결과 합침
    merged = concatenate([conv1, conv2, conv3, conv4], axis=1)

    hidden1 = Dense(1024)(merged)
    hidden1 = ELU()(hidden1)
    hidden1 = BatchNormalization(mode=0)(hidden1)
    hidden1 = Dropout(0.5)(hidden1)

    hidden2 = Dense(256)(hidden1)
    hidden2 = ELU()(hidden2)
    hidden2 = BatchNormalization(mode=0)(hidden2)
    hidden2 = Dropout(0.5)(hidden2)

    hidden3 = Dense(64)(hidden2)
    hidden3 = ELU()(hidden3)
    hidden3 = BatchNormalization(mode=0)(hidden3)
    hidden3 = Dropout(0.5)(hidden3)

    # Output layer (last fully connected layer)
    # 마지막 클래스 결정하는 layer
    output = Dense(21, activation='softmax', name='main_output')(hidden3)

    model = Model(input=[main_input], output=[output])

    return model

In [3]:
# Load data using model preprocessor
x_train, x_test, y_train, y_test = Preprocessor.load_data()

In [4]:
# Define Deep Learning Model
model_name = "1DCNN"
model = conv_fully()

''' Training phrase '''
epochs = 5
batch_size = 64
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1149965 samples, validate on 142131 samples
Epoch 1/5
1149965/1149965 [==============================] - 1089s 947us/step - loss: 0.6817 - accuracy: 0.8179 - categorical_accuracy: 0.7276 - precision: 0.9012 - recall: 0.7566 - fmeasure: 0.8196 - val_loss: 0.3122 - val_accuracy: 0.9068 - val_categorical_accuracy: 0.8230 - val_precision: 0.9340 - val_recall: 0.8865 - val_fmeasure: 0.9094
Epoch 2/5
1149965/1149965 [==============================] - 1333s 1ms/step - loss: 0.3682 - accuracy: 0.8931 - categorical_accuracy: 0.8450 - precision: 0.9329 - recall: 0.8583 - fmeasure: 0.8937 - val_loss: 0.2623 - val_accuracy: 0.9219 - val_categorical_accuracy: 0.8603 - val_precision: 0.9403 - val_recall: 0.9078 - val_fmeasure: 0.9236
Epoch 3/5
1149965/1149965 [==============================] - 1270s 1ms/step - loss: 0.3127 - accuracy: 0.9093 - categorical_accuracy: 0.8699 

In [5]:
model_json = model.to_json()
with open("./saved_model/cnn.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/cnn.h5")